In this notebook, we will be going over what Random Forests are, what they are used for, and how to use them in an R environment.

---

## Why do we need Random Forests?
You might be familiar with the concept of Decision Trees -- a probabilistic predictive model which can be used to classify data in a wide array of applications. Decision Trees are created through observation of data points. A probabilistic model is created by observing the features present in each point labeled a certain class, and then associating probabilities to these features.

<center>
<a title="By Stephen Milborrow (Own work) [CC BY-SA 3.0 (http://creativecommons.org/licenses/by-sa/3.0) or GFDL (http://www.gnu.org/copyleft/fdl.html)], via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File%3ACART_tree_titanic_survivors.png"><img width="320" alt="CART tree titanic survivors" src="https://upload.wikimedia.org/wikipedia/commons/f/f3/CART_tree_titanic_survivors.png"/></a>
<font size="2">*Example of a Decision Tree for the Titanic dataset.*</font>
</center>

Decision Trees are very interesting because one can follow the structure created to understand how the class was inferred. However, this kind of model is not without its own problems. One of the main problems is what is called **overfitting**. Overfitting happens when the process of creating the tree makes it so that the tree is extremely ramified and complex -- this means that the model will not generalize correctly.

This can mean that the data points are too varied, or maybe that there are too many features to be analyzed at once. However, if we cut down the number of data points or features, this might make our model worse. So, we would need another kind of solution for this problem.

## What are Random Forests

Random Forests are one of the proposed solutions. As one might infer from its name, Random Forests are composed of multiple Decision Trees. This makes them part of a family of models -- that are composed of other models working in tandem -- called **ensemble learning models**. The main concept behind Random Forests is that, if you partition the data that would be used to create a single decision tree into different parts, create one tree for each of these partitions, and then use a method to "average" the results of all of these different trees, you should end up with a better model. In the case of trees used for classification, this "average" is the **mode** of the set of trees in the forest. For regression, the "average" is the **mean** of the set of trees in the forest.

The main mechanism behind Random Forests is **bagging**, which is shorthand for **bootstrap aggregating**. Bagging is the concept of randomly sampling some data from a dataset, but **with replacement**. What this means in practice is that there is some amount of data that will be repeated in each partition, and some amount of data that will not be represented in the samples -- about 63% of the unique examples are kept -- this makes it so that the model generated for that bag is able to generalize better to some degree. Each partition of data of our training data for the Random Forest applies this concept.

<center>
<img src="https://ibm.box.com/shared/static/5m7lep2u6fzt6ors1b0kpgv0jtzh3z7z.png" width="480">
<font size="2">*Bagging example. Notice how some data points are repeated -- this is intentional!*</font>
</center>

You might be asking yourself what happens to the data that is not present in the "bags". This data, aptly called *Out-Of-Bag Data*, serves as a kind of **testing data for the generated model** -- which serves as validation that our model works!

Additionally, Random Forests are created using **feature bagging** as well, which makes it so that there are no problems of overfitting due to a large amount of features for a small amount of data. For example, if a few features are very strong predictors, they will be present in a large amount of "bags", and these bags will become correlated. However, this also makes it so that the Random Forest itself does not focus only on what strongly predicts the data that it was fed, making the model generalize better. Traditionally, a dataset with a number $f$ of features will have $\left\lceil{\sqrt[2]{f}}\ \right\rceil$ features in each partition.

<center>
<img src="https://ibm.box.com/shared/static/a4b0d3eg7vtuh8wipj9eo4bat9szow67.png" width="720">
<font size="2">*Example of a Random Forest. Don't forget that the bags can have repeated data points!*</font>
</center>

---

## Using Random Forests in R
Now that you know what Random Forests are, we can move on to use them in R. Conveniently enough, CRAN (R's library repository) has a library ready for us -- aptly named `randomForest`. However, we first need to install it. You can do that by running the code cell below.

In [2]:
install.packages("randomForest")

package 'randomForest' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\satha\AppData\Local\Temp\RtmpoxDHew\downloaded_packages


Once you install it, you won't need to install it again. You just need to load it up whenever you are going to utilize it. You can do that by using the `library` command.

In [4]:
# Load the "randomForest" library into the R context.
library(randomForest)

In [5]:
data(iris)
head(iris)

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa


We can now go ahead and create the model. For this example, we will be using the built-in `iris` dataset. Feel free to try using other datasets!

To create the model, we will use the `randomForest` function. It has a wide array of parameters for customization, but the simplest approach is just to provide it with a `formula` and the dataset to infer the probabilities from. This can be seen in the following code:

In [6]:
# Create the Random Forest model.
# The randomForest function accepts a "formula" structure as its main parameter. In this case, "Species" will be the variable
# to be predicted, while the others will be the predictors.
myLittleForest <- randomForest(Species ~ Sepal.Length + Sepal.Width + Petal.Length + Petal.Width, data = iris)

# Print the summary of our model.
print(myLittleForest)


Call:
 randomForest(formula = Species ~ Sepal.Length + Sepal.Width +      Petal.Length + Petal.Width, data = iris) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 2

        OOB estimate of  error rate: 4%
Confusion matrix:
           setosa versicolor virginica class.error
setosa         50          0         0        0.00
versicolor      0         47         3        0.06
virginica       0          3        47        0.06


In [12]:
(50+47+47)/(50+50+50)

[1] 0.96

As you can see, calling the `print` function on the model we created prints a summary of our model. This summary is quite informative -- it tells us how many trees were created, how many variables were tried at each split, the estimate of the error rate for the Out-of-Bag data (*remember, it works as validation for our model!*), and its confusion matrix.

Another statistic that can be quite informative is the importance of each predictor for the prediction of our data points. This can be done by using the `importance` function, which can be seen in the following code:

In [11]:
print(importance(myLittleForest, type=2))

             MeanDecreaseGini
Sepal.Length         9.391605
Sepal.Width          2.578211
Petal.Length        42.031942
Petal.Width         45.224149


In [18]:
myLittleForest <- randomForest(Species ~ Petal.Length + Petal.Width, data = iris)

# Print the summary of our model.
print(myLittleForest)


Call:
 randomForest(formula = Species ~ Petal.Length + Petal.Width,      data = iris) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 1

        OOB estimate of  error rate: 4.67%
Confusion matrix:
           setosa versicolor virginica class.error
setosa         50          0         0        0.00
versicolor      0         47         3        0.06
virginica       0          4        46        0.08


In [19]:
(50+47+48)/(50+50+50)

[1] 0.9666667

In this case, it seems that the petal length of the flowers is the main difference between species (*the larger the Gini coefficient is, the more different each data point is in terms of that variable*).

---

This is the end of the Random Forests in R notebook. Hopefully, you now understand how Random Forests are structured, how they work and how to utilize them in R. Thank you for reading this notebook, and good luck on your studies.

